In [1]:
import pandas as pd
from pandas import Series
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
%matplotlib inline
import random
import os
import sys
import shutil
import csv
import re
from sklearn.model_selection import KFold,train_test_split,StratifiedKFold

mimic3_path="/home/rafiparvez1706/mimic"
output_path ="../data/root"
variable_map_file='../resources/itemid_to_variable_map.csv'
icu_stays_root_path=output_path

In [1]:
# read all stays data
stays = pd.read_csv('../data/clean_readm_details.csv')

NameError: name 'pd' is not defined

In [3]:
sum(stays.IsReadmitted==1)*100/len(stays)

11.778741865509762

In [4]:
stays.head()

,subject_id,hadm_id,icustay_id,last_wardid,first_careunit,last_careunit,age,gender,marital_status,insurance,IsReadmitted
0,3,145834,211552,12,MICU,MICU,76.526788,M,MARRIED,Medicare,0
1,4,185777,294638,52,MICU,MICU,47.845044,F,SINGLE,Private,0
2,6,107064,228232,33,SICU,SICU,65.940670,F,MARRIED,Medicare,0
3,9,150750,220597,15,MICU,MICU,41.790226,M,NaN,Medicaid,0
4,11,194540,229441,57,SICU,SICU,50.148292,F,MARRIED,Private,0


In [5]:
def break_up_stays_by_icustayids(stays, output_path, verbose=1):
    icustay_ids = stays.icustay_id.unique()
    for i, icustay_id in enumerate(icustay_ids):
        if verbose:
            sys.stdout.write('\ricustay_id {0} of {1}...'.format(i+1, icustay_ids.shape[0]))
        dn = os.path.join(output_path, str(icustay_id))
        if os.path.exists(dn):
            shutil.rmtree(dn)
        os.makedirs(dn)        
        stays.loc[stays.icustay_id == icustay_id].to_csv(os.path.join(dn, 'stays.csv'), index=False)

In [6]:
'''
break_up_stays_by_icustayids(stays,output_path)
'''

'\nbreak_up_stays_by_icustayids(stays,output_path)\n'

In [7]:
def read_events_table_by_row(mimic3_path, table):
    nb_rows = { 'chartevents': 263201376, 'labevents': 27872576, 'outputevents': 4349340 }
    reader = csv.DictReader(open(os.path.join(mimic3_path, table.upper() + '.csv'), 'r'))
    for i,row in enumerate(reader):
        if 'ICUSTAY_ID' not in row:
            row['ICUSTAY_ID'] = ''
        yield row, i, nb_rows[table.lower()]
        
def read_events_table_and_break_up_by_icu_stay(mimic3_path, table, output_path,items_to_keep=None, icu_stay_ids_to_keep=None, verbose=1):
    obs_header = [ 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUEUOM' ]
    if items_to_keep is not None:
        items_to_keep = set([ str(s) for s in items_to_keep ])
    if icu_stay_ids_to_keep is not None:
        icu_stay_ids_to_keep = set([ str(i) for i in icu_stay_ids_to_keep ])

    class DataStats(object):
        def __init__(self):
            self.curr_icu_stay_id = ''
            self.last_write_no = 0
            self.last_write_nb_rows = 0
            self.last_write_icu_stay_id = ''
            self.curr_obs = []

    data_stats = DataStats()

    def write_current_observations():
        data_stats.last_write_no += 1
        data_stats.last_write_nb_rows = len(data_stats.curr_obs)
        data_stats.last_write_icu_stay_id = data_stats.curr_icu_stay_id
        dn = os.path.join(output_path, str(data_stats.curr_icu_stay_id))
        try:
            os.makedirs(dn)
        except:
            pass
        fn = os.path.join(dn, 'events.csv')
        if not os.path.exists(fn) or not os.path.isfile(fn):
            f = open(fn, 'w')
            f.write(','.join(obs_header) + '\n')
            f.close()
        w = csv.DictWriter(open(fn, 'a'), fieldnames=obs_header, quoting=csv.QUOTE_MINIMAL)
        w.writerows(data_stats.curr_obs)
        data_stats.curr_obs = []
    
    for row, row_no, nb_rows in read_events_table_by_row(mimic3_path, table):
        if verbose and (row_no % 100000 == 0):
            if data_stats.last_write_no != '':
                sys.stdout.write('\rprocessing {0}: ROW {1} of {2}...last write '
                                 '({3}) {4} rows for icu_stay {5}'.format(table, row_no, nb_rows,
                                                                         data_stats.last_write_no,
                                                                         data_stats.last_write_nb_rows,
                                                                         data_stats.last_write_icu_stay_id))
            else:
                sys.stdout.write('\rprocessing {0}: ROW {1} of {2}...'.format(table, row_no, nb_rows))
        
        if (icu_stay_ids_to_keep is not None and row['ICUSTAY_ID'] not in icu_stay_ids_to_keep):
            continue
        if (items_to_keep is not None and row['ITEMID'] not in items_to_keep):
            continue
        
        row_out = { 'SUBJECT_ID': row['SUBJECT_ID'],
                    'HADM_ID': row['HADM_ID'],
                    'ICUSTAY_ID': '' if 'ICUSTAY_ID' not in row else row['ICUSTAY_ID'],
                    'CHARTTIME': row['CHARTTIME'],
                    'ITEMID': row['ITEMID'],
                    'VALUE': row['VALUE'],
                    'VALUEUOM': row['VALUEUOM'] }
        if data_stats.curr_icu_stay_id != '' and data_stats.curr_icu_stay_id != row['ICUSTAY_ID']:
            write_current_observations()
        data_stats.curr_obs.append(row_out)
        data_stats.curr_icu_stay_id = row['ICUSTAY_ID']
        
    if data_stats.curr_icu_stay_id != '':
        write_current_observations()

    if verbose and (row_no % 100000 == 0):
        sys.stdout.write('\rprocessing {0}: ROW {1} of {2}...last write '
                         '({3}) {4} rows for icu_stay {5}...DONE!\n'.format(table, row_no, nb_rows,
                                                                 data_stats.last_write_no,
                                                                 data_stats.last_write_nb_rows,
                                                                 data_stats.last_write_icu_stay_id))

In [8]:
#Add events details to each icu stay id
'''
items_to_keep=None
icu_stay_ids=None

for table in ['CHARTEVENTS', 'LABEVENTS', 'OUTPUTEVENTS']: 
    read_events_table_and_break_up_by_icu_stay(mimic3_path, table, output_path,
                                               items_to_keep=items_to_keep,icu_stay_ids_to_keep=icu_stay_ids, verbose=1)
                                               '''

"\nitems_to_keep=None\nicu_stay_ids=None\n\nfor table in ['CHARTEVENTS', 'LABEVENTS', 'OUTPUTEVENTS']: \n    read_events_table_and_break_up_by_icu_stay(mimic3_path, table, output_path,\n                                               items_to_keep=items_to_keep,icu_stay_ids_to_keep=icu_stay_ids, verbose=1)\n                                               "

In [9]:
#events_sample = pd.read_csv('../data/root/226799/events.csv')

## Now creating time-series data

In [18]:
def read_itemid_to_variable_map(fn, variable_column='LEVEL2'):
    var_map = pd.read_csv(fn, index_col=None).fillna('').astype(str)

    var_map.COUNT = var_map.COUNT.astype(int)

    var_map = var_map.loc[(var_map[variable_column] != '') & (var_map.COUNT>0)]
    var_map = var_map.loc[(var_map.STATUS == 'ready')]
    var_map.ITEMID = var_map.ITEMID.astype(int)
    var_map = var_map[[variable_column, 'ITEMID', 'MIMIC LABEL']].set_index('ITEMID')
    return var_map.rename({variable_column: 'VARIABLE', 'MIMIC LABEL': 'MIMIC_LABEL'}, axis=1)

In [20]:
var_map = read_itemid_to_variable_map(variable_map_file)
variables = var_map.VARIABLE.unique()
variables

array(['Capillary refill rate', 'Diastolic blood pressure',
       'Fraction inspired oxygen', 'Glascow coma scale eye opening',
       'Glascow coma scale motor response', 'Glascow coma scale total',
       'Glascow coma scale verbal response', 'Glucose', 'Heart Rate',
       'Height', 'Mean blood pressure', 'Oxygen saturation', 'pH',
       'Respiratory rate', 'Systolic blood pressure', 'Temperature',
       'Weight'], dtype=object)

In [ ]:
#db Connection String
# Create a database connection
user = 'postgres'
host = 'xx.xxx.xx.xxx'
dbname = 'mimic'
schema = 'mimiciii'
password ='xxxx'
port = '5432'

In [12]:
g_map = { 'F': 1, 'M': 2, 'OTHER': 3, '': 0 }
def transform_gender(gender_series):
    global g_map
    return gender_series.fillna('').apply(lambda s: g_map[s] if s in g_map else g_map['OTHER'])

In [13]:
# SBP: some are strings of type SBP/DBP
def clean_sbp(df):
    v = df.VALUE.astype(str)
    idx = v.apply(lambda s: '/' in s)
    v.loc[idx] = v.loc[idx].apply(lambda s: re.match('^(\d+)/(\d+)$', s).group(1))
    return v.astype(float)

def clean_dbp(df):
    v = df.VALUE.astype(str)
    idx = v.apply(lambda s: '/' in s)
    v.loc[idx] = v.loc[idx].apply(lambda s: re.match('^(\d+)/(\d+)$', s).group(2))
    return v.astype(float)

# CRR: strings with brisk, <3 normal, delayed, or >3 abnormal
def clean_crr(df):
    v = Series(np.zeros(df.shape[0]), index=df.index)
    v[:] = np.nan
    
    # when df.VALUE is empty, dtype can be float and comparision with string
    # raises an exception, to fix this we change dtype to str
    df.VALUE = df.VALUE.astype(str)
    
    v.loc[(df.VALUE == 'Normal <3 secs') | (df.VALUE == 'Brisk')] = 0
    v.loc[(df.VALUE == 'Abnormal >3 secs') | (df.VALUE == 'Delayed')] = 1
    return v

# FIO2: many 0s, some 0<x<0.2 or 1<x<20
def clean_fio2(df):
    v = df.VALUE.astype(float)
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'torr' not in s.lower()) & (v>1.0)
    v.loc[idx] = v.loc[idx] / 100.
    return v

# GLUCOSE, PH: sometimes have ERROR as value
def clean_lab(df):
    v = df.VALUE
    idx = v.apply(lambda s: type(s) is str and not re.match('^(\d+(\.\d*)?|\.\d+)$', s))
    v.loc[idx] = np.nan
    return v.astype(float)

# O2SAT: small number of 0<x<=1 that should be mapped to 0-100 scale
def clean_o2sat(df):
    # change "ERROR" to NaN
    v = df.VALUE
    idx = v.apply(lambda s: type(s) is str and not re.match('^(\d+(\.\d*)?|\.\d+)$', s))
    v.loc[idx] = np.nan
    
    v = v.astype(float)
    idx = (v<=1)
    v.loc[idx] = v.loc[idx] * 100.
    return v

# Temperature: map Farenheit to Celsius, some ambiguous 50<x<80
def clean_temperature(df):
    v = df.VALUE.astype(float)
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'F' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'F' in s.lower()) | (v >= 79)
    v.loc[idx] = (v.loc[idx] - 32) * 5. / 9
    return v

# Weight: some really light/heavy adults: <50 lb, >450 lb, ambiguous oz/lb
# Children are tough for height, weight
def clean_weight(df):
    v = df.VALUE.astype(float)
    # ounces
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'oz' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'oz' in s.lower())
    v.loc[idx] = v.loc[idx] / 16.
    # pounds
    idx = idx | df.VALUEUOM.fillna('').apply(lambda s: 'lb' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'lb' in s.lower())
    v.loc[idx] = v.loc[idx] * 0.453592
    return v

# Height: some really short/tall adults: <2 ft, >7 ft)
# Children are tough for height, weight
def clean_height(df):
    v = df.VALUE.astype(float)
    idx = df.VALUEUOM.fillna('').apply(lambda s: 'in' in s.lower()) | df.MIMIC_LABEL.apply(lambda s: 'in' in s.lower())
    v.loc[idx] = np.round(v.loc[idx] * 2.54)
    return v

# ETCO2: haven't found yet
# Urine output: ambiguous units (raw ccs, ccs/kg/hr, 24-hr, etc.)
# Tidal volume: tried to substitute for ETCO2 but units are ambiguous
# Glascow coma scale eye opening
# Glascow coma scale motor response
# Glascow coma scale total
# Glascow coma scale verbal response
# Heart Rate
# Respiratory rate
# Mean blood pressure
clean_fns = {
    'Capillary refill rate': clean_crr,
    'Diastolic blood pressure': clean_dbp,
    'Systolic blood pressure': clean_sbp,
    'Fraction inspired oxygen': clean_fio2,
    'Oxygen saturation': clean_o2sat,
    'Glucose': clean_lab,
    'pH': clean_lab,
    'Temperature': clean_temperature,
    'Weight': clean_weight,
    'Height': clean_height
}
def clean_events(events):
    global cleaning_fns
    for var_name, clean_fn in clean_fns.items():
        idx = (events.VARIABLE == var_name)
        try:
            events.VALUE.loc[idx] = clean_fn(events.loc[idx])
        except Exception as e:
            print("Exception in clean_events:", clean_fn.__name__, e)
            print("number of rows:", np.sum(idx))
            print("values:", events.loc[idx])
            exit()
    return events.loc[events.VALUE.notnull()]

In [14]:
def convert_events_to_timeseries(events, variable_column='VARIABLE', variables=[]):
    metadata = events[['CHARTTIME']].sort_values(by=['CHARTTIME'])\
                    .drop_duplicates(keep='first').set_index('CHARTTIME')
    timeseries = events[['CHARTTIME', variable_column, 'VALUE']]\
                    .sort_values(by=['CHARTTIME', variable_column, 'VALUE'], axis=0)\
                    .drop_duplicates(subset=['CHARTTIME', variable_column], keep='last')
    timeseries = timeseries.pivot(index='CHARTTIME', columns=variable_column, values='VALUE').merge(metadata, left_index=True, right_index=True)\
                    .sort_index(axis=0).reset_index()
    for v in variables:
        if v not in timeseries:
            timeseries[v] = np.nan
    return timeseries

In [15]:
def add_hours_elpased_to_events(events, dt, remove_charttime=True):
    events['HOURS'] = (events.CHARTTIME - dt).apply(lambda s: s / np.timedelta64(1, 's')) / 60./60
    if remove_charttime:
        del events['CHARTTIME']
    return events

In [21]:
pd.options.mode.chained_assignment = None  # default='warn'
'''
def read_events(icu_stay_path, remove_null=True):
    events = pd.read_csv(os.path.join(icu_stay_path, 'events.csv'), index_col=None)
    if remove_null:
        events = events.loc [events.VALUE.notnull()]
    events.CHARTTIME = pd.to_datetime(events.CHARTTIME)
    events.HADM_ID = events.HADM_ID.fillna(value=-1).astype(int)
    events.ICUSTAY_ID = events.ICUSTAY_ID.fillna(value=-1).astype(int)
    events.VALUEUOM = events.VALUEUOM.fillna('').astype(str)
#    events.sort_values(by=['CHARTTIME', 'ITEMID', 'ICUSTAY_ID'], inplace=True)
    return events

nb_rows = len(stays.icustay_id.unique())

buggy_ids=[];

for ctr,icu_stay_id in enumerate(stays.icustay_id.unique()):
    sys.stdout.write('\rprocessing {0} of {1} ICUSTAY_IDs...'.format(ctr, nb_rows))
#for icu_stay_id in [236962]:
    icu_stay_dir = str(icu_stay_id)
    dn = os.path.join(icu_stays_root_path, icu_stay_dir)
    try:
        #icu_stay_id = int(icu_stay_dir)
        if not os.path.isdir(dn):
            raise Exception
    except:
        continue
    #print('ICUSTAY_ID {}: '.format(icu_stay_id))
    
    #sys.stdout.write('Subject {}: '.format(icu_stay_id))
    #sys.stdout.flush()
    
    try:
        icustays = pd.read_csv(os.path.join(dn, 'stays.csv'))
        icuevents = read_events(dn)
    except:
        #print('For '+icu_stay_dir+' error reading from disk!\n')
        buggy_ids.append(icu_stay_id)
        continue
    #else:
        #print('got {0} stays, {1} events...'.format(icustays.shape[0], icuevents.shape[0]))
    icustays.gender = transform_gender(icustays.gender)
    icuevents =  icuevents.merge(var_map, left_on='ITEMID', right_index=True)
    if len(icuevents)==0:
        buggy_ids.append(icu_stay_id)
        continue
    icuevents = clean_events(icuevents)
    timeseries = convert_events_to_timeseries(icuevents, variables=variables)
    
    episode = add_hours_elpased_to_events(timeseries, timeseries.CHARTTIME[0]).set_index('HOURS').sort_index(axis=0)
    columns = list(episode.columns)
    columns_sorted = sorted(columns, key=(lambda x: "" if x == "Hours" else x))
    episode = episode[columns_sorted]
    
    episode.to_csv(os.path.join(dn, 'episode_timeseries.csv'), index_label='Hours')
    
    
    '''

'\ndef read_events(icu_stay_path, remove_null=True):\n    events = pd.read_csv(os.path.join(icu_stay_path, \'events.csv\'), index_col=None)\n    if remove_null:\n        events = events.loc [events.VALUE.notnull()]\n    events.CHARTTIME = pd.to_datetime(events.CHARTTIME)\n    events.HADM_ID = events.HADM_ID.fillna(value=-1).astype(int)\n    events.ICUSTAY_ID = events.ICUSTAY_ID.fillna(value=-1).astype(int)\n    events.VALUEUOM = events.VALUEUOM.fillna(\'\').astype(str)\n#    events.sort_values(by=[\'CHARTTIME\', \'ITEMID\', \'ICUSTAY_ID\'], inplace=True)\n    return events\n\nnb_rows = len(stays.icustay_id.unique())\n\nbuggy_ids=[];\n\nfor ctr,icu_stay_id in enumerate(stays.icustay_id.unique()):\n    sys.stdout.write(\'\rprocessing {0} of {1} ICUSTAY_IDs...\'.format(ctr, nb_rows))\n#for icu_stay_id in [236962]:\n    icu_stay_dir = str(icu_stay_id)\n    dn = os.path.join(icu_stays_root_path, icu_stay_dir)\n    try:\n        #icu_stay_id = int(icu_stay_dir)\n        if not os.path.isdir(

In [25]:
#pd.Series(buggy_ids).to_csv('../data/buggy_ids.csv', index=False)
#working = list(set(stays.icustay_id.unique())- set(buggy_ids) )
#pd.Series(working).to_csv('../data/working_ids.csv', index=False)